In [7]:
import pandas as pd
import numpy as np
import csv
import itertools

def LFOC(cut,CAPACITIES):
    #This finds all local flows over the cut
    names=[]
    caps=[]
    back_names=[]
    for arc in cut.columns:
        #search through each arc
        if cut[arc].iloc[0]!=0:
            #If the arc is included in the cut, we add its capacities to the list of included arc capacities
            caps.append(list(CAPACITIES[arc]))
            if cut[arc].iloc[0]!=1:
                #If the arc is a backward arc, we make a note of this (we will subtract the flows along backward arcs
                #when computing flow values)
                    back_names.append(arc)
            names.append(arc)
            #Note all included arcs, forward and backward
    df=pd.DataFrame(data=itertools.product(*caps), columns=names)
    #df stores the cartesian product of capacities
    df2=df.copy()
    for arc in back_names:
        #we apply the -1 multiplier to backward arcs in a copy of df, then take the sum to get proper addition 
        #and subtraction
        df2[arc]=df2[arc]*-1
    df['Flow Value']=df2.sum(axis=1)
    #Returns a list of all local flows over the cut, along with flow value
    return df
    
def ALL_LFOC(CUTS,CAPACITIES):
    ALFOC={} #dictionary which associates cuts with LFOC
    for cut_number in list(CUTS.index):
        cut=pd.DataFrame(CUTS.iloc[cut_number]).T
        #Pulls row vector of particular cut out of CUTS as a dataframe
        lfoc=LFOC(cut,CAPACITIES)
        #Find collection of local flows over cuts
        ALFOC[cut_number]=lfoc
        #Link cut number to lfoc in dictionary
    return ALFOC

def AUGMENT(lfoc,arc_names):
    #Run on initial  cut to get dataframe with all column names.  lfoc is the collection of local flows over the
    #first cut (from the LFOC function), and arc_names is the list of arc names
    for name in arc_names:
        if not name in lfoc.columns:
            #If we hit an arc not in our cut, add the arc as a column header, give NaN flow values
            lfoc[name]=np.nan



def GLUEING(df,lfoc):
    glued_data=pd.DataFrame(columns=df.columns)
    #The dataframe df holds all local flows over the cuts covered thus far, with NA for unused arcs. Local flows over the
    #cut to be added are stored on the dataframe lfoc. For each row in lfoc, this function searches for all rows of df 
    #which are either equal to the entry in lfoc or are NA. We create a row in the glued_data dataframe for each such 
    #row discovered in the df, adding the gluing of the two (that is, the df row with NA replaced by values from the lfoc
    #row where applicable).
    for flow_num in lfoc.index:
        for row_num in df.index:
            #Iterate through all pairs of local flows in lfoc and df
            include=1
            #Indicates if the pair is consistent on overlap. Default yes.
            row=df.iloc[row_num]
            for column in lfoc.columns:
                #Check each column in lfoc for consistency (note this is a subset of df columns)
                if not np.array_equal(lfoc[column][flow_num],df[column][row_num],equal_nan=True):
                    #If the flow values are not equal
                    if type(df[column][row_num])==np.ndarray:
                        #And if this is not because df contains a NaN here (defined flows have a data type of np.ndarray)
                        include=0
                        #We don't include the pairing
                        break
                        #We also don't need to continue searching through lfoc's columns, so we break
            if include==1:
                #As long as we haven't found any inconsistencies
                for column in lfoc.columns:
                    if not type(row[column])==np.ndarray:
                        #We replace any NaNs in df which overlapped with the corresponding entry from lfoc
                        row[column]=lfoc[column][flow_num]
                glued_data=glued_data.append(row, ignore_index=True)
                #Append this data to the glued_data table
    return(glued_data)
    #Return the data table of all local flows over the expanded cut set

def FLOW_VALUES(CUTS,CAPACITIES,printing=False):
    #This is the wrapper function for the Max Flow Min Cut algorithm here. CUTS is a dataframe with column names given
    #by the arcs in the network, rows corresponding to cuts, and entries indicating if the arc corresponding to the 
    #column is in the cut corresponding to the row. A 1 indicates the presence of the forward arc, a 0 indicates the
    #absense of the arc, and a -1 indicates the presence of the backward arc.
    #CAPACITIES is a dictionary linking arc names to a numpy array of lists of allowed flows through the arc.
    #For instance, in a two-commodity network, we might have 'a1':np.array([[0,0],[0,1],[1,0],[1,1],[2,0],[2,1]])
    #as one entry, meaning [0,0], [0,1],  ... are legal flow pairings of commodities x_1,x_2 along arc a1.
    #printing=True prints code for latex tables containing lfocs and the output of gluing operations.
    names=['Flow Value']
    names.extend(list(CAPACITIES.keys()))
    #names now contains an entry for 'Flow Value' and all arcs.
    alfoc=ALL_LFOC(CUTS,CAPACITIES)
    #alfoc is a dictionary linking cut numbers to associated lfocs
    AUGMENT(alfoc[0],CAPACITIES.keys())
    #Make sure the first cut's lfoc contains all column names, NaN for unused arcs
    glued_data=alfoc[0]
    #
    if printing:
        print('\begin{table}')
        print('\centering')
        print('caption{Flows over cut 0}')
        print(glued_data[names].to_latex())
        print('\end{table}')
    toglue=list(alfoc.keys())[1::]
    #Start with alfoc[0] as glued data, then iterate through cuts, glueing new cut to glued_data at each step
    i=1
    #count for printing
    for cut in toglue:
        glued_data=GLUEING(glued_data,alfoc[cut])
        #Glues new cut to glued_data
        if printing:
            print('\begin{table}')
            print('Gluing cut '+str(i))
            print('\centering')
            print(glued_data[names].to_latex())
            print('\end{table}')
    return(glued_data[names])
                    



    

In [8]:
#INPUT

d={'a1':[1,0,1,0],'a2':[1,1,0,0],'a3':[0,1,-1,0],'a4':[0,1,0,1],'a5':[0,0,1,1]}
CUTS=pd.DataFrame(data=d)
#Dataframe with arcs as columns, cuts as rows, and weights (that is, -1 for the reverse arc, 0 for unused, 1 for forward arc)

CAPACITIES={'a1':np.array([[0,0],[0,1],[1,0],[1,1],[2,0],[2,1]]),'a2':np.array([[0,0],[0,1],[1,0]]),'a3':np.array([[0,0],[0,1],[0,2],[1,0],[1,1],[1,2]]),'a4':np.array([[0,0],[0,1]]),'a5':np.array([[0,0],[0,1],[0,2],[1,0],[1,1],[1,2],[2,0],[2,1],[2,2]])}
#Dictionary giving numpy array of all k-tuples which fit through associated arc. For instance, if an arc a1 had capacity 0≤x≤2, 0≤y≤2
#this would be a1:[(0,0),(0,1),(0,2),(1,0),(1,1),(1,2),(2,0),(2,1),(2,2)]

In [9]:
#Table of Global Flows

FLOW_VALUES(CUTS,CAPACITIES)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Flow Value,a1,a2,a3,a4,a5
0,"[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]"
1,"[0, 1]","[0, 0]","[0, 1]","[0, 0]","[0, 0]","[0, 1]"
2,"[0, 1]","[0, 1]","[0, 0]","[0, 1]","[0, 0]","[0, 1]"
3,"[0, 2]","[0, 1]","[0, 1]","[0, 1]","[0, 0]","[0, 2]"
4,"[1, 0]","[0, 0]","[1, 0]","[0, 0]","[0, 0]","[1, 0]"
5,"[1, 0]","[1, 0]","[0, 0]","[1, 0]","[0, 0]","[1, 0]"
6,"[1, 1]","[0, 1]","[1, 0]","[0, 1]","[0, 0]","[1, 1]"
7,"[1, 1]","[1, 0]","[0, 1]","[1, 0]","[0, 0]","[1, 1]"
8,"[1, 1]","[1, 1]","[0, 0]","[1, 1]","[0, 0]","[1, 1]"
9,"[1, 2]","[1, 1]","[0, 1]","[1, 1]","[0, 0]","[1, 2]"
